In [12]:
import pandas as pd
import os
from utils import construct_path

pd.set_option('display.max_columns', None)

In [13]:
app_train = pd.read_csv(construct_path('data/application_train.csv'))
app_test = pd.read_csv(construct_path('data/application_test.csv'))

# Location of other kaggle dfs
data_folder = construct_path('data/aux_data')

# Create a dict, mapping file names to raw and aggregated dfs
dfs = {}
for file in os.listdir(data_folder):
    if file.endswith('.csv'):
        name = file.replace('.csv', '')
        raw_df = pd.read_csv(construct_path(f"{data_folder}/{file}"))
        dfs[name] = {'raw': raw_df, 'agg': pd.DataFrame()}


In [14]:
dfs.keys()

dict_keys(['bureau', 'bureau_balance', 'credit_card_balance', 'installments_payments', 'POS_CASH_balance', 'previous_application'])

In [15]:
def print_df_mem_usage(df):
    memory_in_bytes = df.memory_usage(deep=True).sum()
    # Convert to MG
    memory_in_mb = memory_in_bytes / (1024 ** 2)
    print(f"Memory Usage: {memory_in_mb:.2f} MB")

def explore_id_distribution(df, id_column='SK_ID_CURR'):
    # Count occurrences of each SK_ID_CURR
    count_series = df[id_column].value_counts()

    print(tuple(f"{key}{value}" for key, value in dict(zip(("Rows: ", "Cols: "), ('{:,}'.format(x) for x in df.shape))).items()))
    print(df.columns.to_list())
    print_df_mem_usage(df)
    print(f"Total unique {id_column}: {'{:,}'.format(count_series.shape[0])}")
    print(f"Average rows per {id_column}: {'{:.2f}'.format(count_series.mean())}")
    print(f"Max rows per {id_column}: {'{:,}'.format(count_series.max())}")
    print(f"Min rows per {id_column}: {'{:,}'.format(count_series.min())}")
    print(f"Rows with more than one {id_column}: {'{:,}'.format((count_series > 1).sum())}")
    # border
    print("-" * 40)
    
    return count_series

In [16]:

for name, df in dfs.items():
    print(f"DataFrame: {name}")
    if name == "bureau_balance":
        explore_id_distribution(df["raw"], id_column="SK_ID_BUREAU")
    else:
        explore_id_distribution(df["raw"])

DataFrame: bureau
('Rows: 1,716,428', 'Cols: 17')
['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']
Memory Usage: 472.82 MB
Total unique SK_ID_CURR: 305,811
Average rows per SK_ID_CURR: 5.61
Max rows per SK_ID_CURR: 116
Min rows per SK_ID_CURR: 1
Rows with more than one SK_ID_CURR: 264,291
----------------------------------------
DataFrame: bureau_balance
('Rows: 27,299,925', 'Cols: 3')
['SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS']
Memory Usage: 1718.33 MB
Total unique SK_ID_BUREAU: 817,395
Average rows per SK_ID_BUREAU: 33.40
Max rows per SK_ID_BUREAU: 97
Min rows per SK_ID_BUREAU: 1
Rows with more than one SK_ID_BUREAU: 811,206
----------------------------------------
DataFrame: credit_card_balance
('

In [ ]:
def aggregate_df(df, df_name, key_column='SK_ID_CURR', drop_columns=[]):
    """
    Aggregates a DataFrame by applying min, max, mean, std, sum to all numeric columns,
    and one hot encodes and sums categorical columns.
    """

    # Add SK_ID_CURR to bureau_balance by merging with bureau using SK_ID_BUREAU
    if df_name == 'bureau_balance' and 'SK_ID_CURR' not in df.columns:
        df = df.merge(dfs['bureau']['raw'][['SK_ID_BUREAU', 'SK_ID_CURR']], on='SK_ID_BUREAU', how='left')

    # Detect categorical columns and one hot encode them, if any
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    if categorical_columns.any():
        df_encoded = pd.get_dummies(df, columns=categorical_columns)
    else:
        df_encoded = df

    # Identify numeric columns
    numeric_columns = df_encoded.select_dtypes(include='number').columns.tolist()

    # Define aggregation functions
    agg_funcs = {
        # Apply only sum to categorical, and basic summary stats to numeric
        col: ['min', 'max', 'mean', 'std', 'sum'] if col in numeric_columns else 'sum'
        # Exclude id columns from aggregations
        for col in df_encoded.columns if col != key_column and not col.startswith("SK_ID")
    }

    # Apply aggregations
    agg_df = df_encoded.groupby(key_column).agg(agg_funcs).reset_index()
    
    # Flatten columns
    agg_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in agg_df.columns]
    
    # Drop unnecessary columns, if specified
    agg_df.drop(columns=drop_columns, inplace=True, errors='ignore')
    
    # Rename key column to avoid unwanted suffix
    agg_df.rename(columns={f"{key_column}_": key_column}, inplace=True)

    return agg_df

for name, df in dfs.items():
    print(f"aggregating {name}")
    df['agg'] = aggregate_df(df['raw'], name, key_column='SK_ID_CURR')
    # explore_id_distribution(df["agg"])


aggregating bureau
aggregating bureau_balance
aggregating credit_card_balance
aggregating installments_payments
aggregating POS_CASH_balance
aggregating previous_application


In [57]:
dfs['bureau_balance']['agg']

,SK_ID_CURR,MONTHS_BALANCE_min,MONTHS_BALANCE_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_sum,STATUS_0_sum,STATUS_1_sum,STATUS_2_sum,STATUS_3_sum,STATUS_4_sum,STATUS_5_sum,STATUS_C_sum,STATUS_X_sum
0,100001.0,-51,0,-16.279070,12.430533,-2800,31,1,0,0,0,0,110,30
1,100002.0,-47,0,-24.554545,11.104662,-2701,45,27,0,0,0,0,23,15
2,100005.0,-12,0,-4.333333,3.812261,-91,14,0,0,0,0,0,5,2
3,100010.0,-90,-2,-46.000000,28.663173,-3312,20,0,0,0,0,0,52,0
4,100013.0,-68,0,-29.373913,18.318369,-6756,79,7,0,0,0,0,103,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134537,456247.0,-81,0,-27.281250,21.366535,-8730,66,0,0,0,0,0,219,35
134538,456250.0,-32,0,-14.149425,8.678400,-1231,12,0,0,0,0,0,25,50
134539,456253.0,-30,0,-14.282051,8.721600,-1671,47,0,0,0,0,0,57,13
134540,456254.0,-36,0,-18.000000,10.824355,-666,8,0,0,0,0,0,29,0


In [ ]:
# Confirm that there are not multiple rows per sk_id_curr
for name, df in dfs.items():
    print(f"DataFrame: {name}")
    explore_id_distribution(df["agg"])
    # print(df["agg"].columns)

DataFrame: bureau
('Rows: 305,811', 'Cols: 84')
['SK_ID_CURR', 'DAYS_CREDIT_min', 'DAYS_CREDIT_max', 'DAYS_CREDIT_mean', 'DAYS_CREDIT_std', 'DAYS_CREDIT_sum', 'CREDIT_DAY_OVERDUE_min', 'CREDIT_DAY_OVERDUE_max', 'CREDIT_DAY_OVERDUE_mean', 'CREDIT_DAY_OVERDUE_std', 'CREDIT_DAY_OVERDUE_sum', 'DAYS_CREDIT_ENDDATE_min', 'DAYS_CREDIT_ENDDATE_max', 'DAYS_CREDIT_ENDDATE_mean', 'DAYS_CREDIT_ENDDATE_std', 'DAYS_CREDIT_ENDDATE_sum', 'DAYS_ENDDATE_FACT_min', 'DAYS_ENDDATE_FACT_max', 'DAYS_ENDDATE_FACT_mean', 'DAYS_ENDDATE_FACT_std', 'DAYS_ENDDATE_FACT_sum', 'AMT_CREDIT_MAX_OVERDUE_min', 'AMT_CREDIT_MAX_OVERDUE_max', 'AMT_CREDIT_MAX_OVERDUE_mean', 'AMT_CREDIT_MAX_OVERDUE_std', 'AMT_CREDIT_MAX_OVERDUE_sum', 'CNT_CREDIT_PROLONG_min', 'CNT_CREDIT_PROLONG_max', 'CNT_CREDIT_PROLONG_mean', 'CNT_CREDIT_PROLONG_std', 'CNT_CREDIT_PROLONG_sum', 'AMT_CREDIT_SUM_min', 'AMT_CREDIT_SUM_max', 'AMT_CREDIT_SUM_mean', 'AMT_CREDIT_SUM_std', 'AMT_CREDIT_SUM_sum', 'AMT_CREDIT_SUM_DEBT_min', 'AMT_CREDIT_SUM_DEBT_max', '

### Merge

In [59]:
merged_df_train = app_train
merged_df_test = app_test

join_key = 'SK_ID_CURR'
for name, df in dfs.items():
    print(f"merging {name}")
    suffix = f"_{name}"
    merged_df_train = merged_df_train.join(dfs[name]['agg'].set_index(join_key), on=join_key, how='left', rsuffix=suffix)
    merged_df_test = merged_df_test.join(dfs[name]['agg'].set_index(join_key), on=join_key, how='left', rsuffix=suffix)

merging bureau
merging bureau_balance
merging credit_card_balance
merging installments_payments
merging POS_CASH_balance
merging previous_application


In [60]:
merged_df_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_mean,DAYS_CREDIT_std,DAYS_CREDIT_sum,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_sum,DAYS_CREDIT_ENDDATE_min,DAYS_CREDIT_ENDDATE_max,DAYS_CREDIT_ENDDATE_mean,DAYS_CREDIT_ENDDATE_std,DAYS_CREDIT_ENDDATE_sum,DAYS_ENDDATE_FACT_min,DAYS_ENDDATE_FACT_max,DAYS_ENDDATE_FACT_mean,DAYS_ENDDATE_FACT_std,DAYS_ENDDATE_FACT_sum,AMT_CREDIT_MAX_OVERDUE_min,AMT_CREDIT_MAX_OVERDUE_max,AMT_CREDIT_MAX_OVERDUE_mean,AMT_CREDIT_MAX_OVERDUE_std,AMT_CREDIT_MAX_OVERDUE_sum,CNT_CREDIT_PROLONG_min,CNT_CREDIT_PROLONG_max,CNT_CREDIT_PROLONG_mean,CNT_CREDIT_PROLONG_std,CNT_CREDIT_PROLONG_sum,AMT_CREDIT_SUM_min,AMT_CREDIT_SUM_max,AMT_CREDIT_SUM_mean,AMT_CREDIT_SUM_std,AMT_CREDIT_SUM_sum,AMT_CREDIT_SUM_DEBT_min,AMT_CREDIT_SUM_DEBT_max,AMT_CREDIT_SUM_DEBT_mean,AMT_CREDIT_SUM_DEBT_std,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_LIMIT_min,AMT_CREDIT_SUM_LIMIT_max,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_LIMIT_std,AMT_CREDIT_SUM_LIMIT_sum,AMT_CREDIT_SUM_OVERDUE_min,AMT_CREDIT_SUM_OVERDUE_max,AMT_CREDIT_SUM_OVERDUE_mean,AMT_CREDIT_SUM_OVERDUE_std,AMT_CREDIT_SUM_OVERDUE_sum,DAYS_CREDIT_UPDATE_min,DAYS_CREDIT_UPDATE_max,DAYS_CREDIT_UPDATE_mean,DAYS_CREDIT_UPDATE_std,DAYS_CREDIT_UPDATE_sum,AMT_ANNUITY_min,AMT_ANNUITY_max,AMT_ANNUITY_mean,AMT_ANNUITY_std,AMT_ANNUITY_sum,CREDIT_ACTIVE_Active_sum,CREDIT_ACTIVE_Bad debt_sum,CREDIT_ACTIVE_Closed_sum,CREDIT_ACTIVE_Sold_sum,CREDIT_CURRENCY_currency 1_sum,CREDIT_CURRENCY_currency 2_sum,CREDIT_CURRENCY_currency 3_sum,CREDIT_CURRENCY_currency 4_sum,CREDIT_TYPE_Another type of loan_sum,CREDIT_TYPE_Car loan_sum,CREDIT_TYPE_Cash loan (non-earmarked)_sum,CREDIT_TYPE_Consumer credit_sum,CREDIT_TYPE_Credit card_sum,CREDIT_TYPE_Interbank credit_sum,CREDIT_TYPE_Loan for

In [61]:
merged_df_train.to_csv(construct_path("data/merged_df_train_raw.csv"), index=False)
merged_df_test.to_csv(construct_path("data/merged_df_test_raw.csv"), index=False)